In [1]:
%%capture
!pip install datasets h5py

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import sys
import optax
import equinox as eqx
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython import display
from functools import partial
from jax.lax import scan, dot_general
from load_ConDiff import load_ConDiff
from architectures import DilResNet, FFNO
from jax import config, random, grad, vmap, jit
from jax.tree_util import tree_map, tree_flatten

config.update("jax_enable_x64", True)

os.environ["CUDA_VISIBLE_DEVICES"] = '2'

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [4]:
def compute_relative_error(model, features, targets, coordinates, n):
    prediction = vmap(model, in_axes=(0, None))(features[n], coordinates)
    relative_error = jnp.linalg.norm((prediction - targets[n]).reshape(n.size, -1), axis=1) / jnp.linalg.norm(targets[n].reshape(n.size, -1), axis=1)
    return relative_error

def train_on_epoch(key, batch_size, model, features, coordinates, targets, opt_state, make_step):
    N_samples = len(features)
    list_of_indices = jnp.linspace(0, N_samples-1, N_samples, dtype=jnp.int64)
    n_batches = N_samples // batch_size
    carry = [model, features, coordinates, targets, opt_state]
    n = random.choice(key, list_of_indices, shape = (n_batches, batch_size))
    data, epoch_loss = scan(make_step, carry, n)
    model = data[0]
    opt_state = data[4]
    return epoch_loss, model, opt_state

def test_on_epoch(key, batch_size, model, features, coordinates, targets):
    N_samples_test = len(features)
    list_of_indices_test = jnp.linspace(0, N_samples_test-1, N_samples_test, dtype=jnp.int64)
    n_batches_test = N_samples_test // batch_size
    n_test = random.choice(key, list_of_indices_test, shape = (n_batches_test, batch_size))
    loss_test_error = lambda a, ind: (None, FFNO.compute_loss(model, features[ind], coordinates, targets[ind]))
    _, test_loss = scan(loss_test_error, None, n_test)
    return jnp.mean(test_loss)

In [5]:
def get_FFNO(key, coordinates, features, grid):
    input = features[0]
    D = len(input.shape[1:])
    
    N_modes = 12
    N_features_out = 1
    
    if grid <= 64:
        N_features_ = 48
    else:
        N_features_ = grid // 4
    N_layers = 4
    N_features = [coordinates.shape[0] + features.shape[1], N_features_, N_features_out]
    
    model = FFNO.FFNO(N_layers, N_features, N_modes, key, D)
    
    N_epoch = 400
    
    if grid <= 64:
        batch_size = 4
    else:
        batch_size = 16
        
    learning_rate = 1e-3
    
    model_data = {
        "model": model
    }
    
    optimization_specification = {
        "learning_rate": learning_rate,
        "compute_loss": FFNO.compute_loss,
        "make_step": FFNO.make_step,
        "N_epochs": N_epoch,
        "batch_size": batch_size
    }
    return model_data, optimization_specification

In [6]:
def train_model(model_data, features, coordinates, targets, optimization_specification):
    model = model_data['model']
    
    history_train = []
    history_test = []
    
    features_train, features_test = features[0], features[1]
    targets_train, targets_test = targets[0], targets[1]

    c = features_train.shape[0] // optimization_specification['batch_size']
    keys = tree_map(lambda x: x * c, np.arange(50, 1000, 50))
    values = [0.5, ] * len(keys)
    dict_lr = dict(zip(keys, values))

    sc = optax.piecewise_constant_schedule(optimization_specification['learning_rate'], dict_lr)
    optimizer = optax.adamw(sc, weight_decay=1e-2)
    opt_state = optimizer.init(eqx.filter(model, eqx.is_array))

    make_step = lambda x, y: optimization_specification['make_step'](x, y, optimizer)

    for it in tqdm(range(optimization_specification['N_epochs'])):
        key = random.PRNGKey(it)
        loss, model, opt_state = train_on_epoch(key, optimization_specification['batch_size'], model, features_train, coordinates, targets_train, opt_state, make_step)
        history_train.append(jnp.mean(loss))

        test_loss = test_on_epoch(key, optimization_specification['batch_size'], model, features_test, coordinates, targets_test)
        history_test.append(test_loss)

    return model, history_train, history_test

def get_datasets(direction_to_save, type_of_pde, grid):
    train_dataset, test_dataset = load_ConDiff(save_dir=direction_to_save, 
                                               pde=type_of_pde, 
                                               grid=grid)
    
    x = jnp.linspace(0, 1, grid)
    y = jnp.linspace(0, 1, grid)
    x_, y_ = jnp.meshgrid(x, y)
    coordinates = jnp.stack([x_, y_], 0)
    
    features_train = tree_map(lambda x: x.reshape(-1, grid, grid), train_dataset[0])
    targets_train = tree_map(lambda x: x.reshape(-1, grid, grid), train_dataset[1])
    
    features_test = tree_map(lambda x: x.reshape(-1, grid, grid), test_dataset[0])
    targets_test = tree_map(lambda x: x.reshape(-1, grid, grid), test_dataset[1])
    
    return [jnp.expand_dims(features_train, 1), jnp.expand_dims(targets_train, 1)], \
           [jnp.expand_dims(features_test, 1), jnp.expand_dims(targets_test, 1)], coordinates

In [7]:
def get_results(key, direction_to_save, type_of_pde, grid):
    keys = random.split(key, 3)
    
    train_dataset, test_dataset, coordinates = get_datasets(direction_to_save, type_of_pde, grid)
    features_train, targets_train = train_dataset[0], train_dataset[1]
    features_test, targets_test = test_dataset[0], test_dataset[1]
    
    model_data, optimization_specification = get_FFNO(keys[0], coordinates, features_train, grid)
    model, history_train, history_test = train_model(model_data, features, coordinates, targets, 
                                                     optimization_specification=optimization_specification)
    
    N_samples_train = features_train.shape[0]
    list_of_indices_train = jnp.linspace(0, N_samples_train-1, N_samples_train, dtype=jnp.int64)
    n_batches_train = N_samples_train // optimization_specification['batch_size']
    n_train = random.choice(keys[1], list_of_indices_train, shape = (n_batches_train, optimization_specification['batch_size']))
    
    N_samples_test = features_test.shape[0]
    list_of_indices_test = jnp.linspace(0, N_samples_test-1, N_samples_test, dtype=jnp.int64)
    n_batches_test = N_samples_test // optimization_specification['batch_size']
    n_test = random.choice(keys[2], list_of_indices_test, shape = (n_batches_test, optimization_specification['batch_size']))
    
    loss_train_error = lambda a, x: (None, compute_relative_error(model, features_train, targets_train, coordinates, x))
    _, train_loss_error = scan(loss_train_error, None, n_train)
    train_error_mean = jnp.mean(train_loss_error.reshape(-1,))

    loss_test_error = lambda a, x: (None, compute_relative_error(model, features_test, targets_test, coordinates, x))
    _, test_loss_error = scan(loss_test_error, None, n_test)
    test_error_mean = jnp.mean(test_loss_error.reshape(-1,))

    train_error_std = jnp.sqrt(jnp.var(test_loss_error.reshape(-1,)))
    test_error_std = jnp.sqrt(jnp.var(test_loss_error.reshape(-1,)))

    model_size = sum(tree_map(jnp.size, tree_flatten(model)[0], is_leaf=eqx.is_array))
    
    data = {
        'history_train': history_train,
        'history_test': history_test,
        'train_error_mean': train_error_mean,
        'test_error_mean': test_error_mean,
        'train_error_std': train_error_std,
        'test_error_std': test_error_std,
        'model_size': model_size
    }
    return data, model, model_data, features_test, targets_test

In [8]:
def train_model(model_data, features, coordinates, targets, optimization_specification):
    model = model_data['model']
    
    history_train = []
    history_test = []
    
    features_train, features_test = features[0], features[1]
    targets_train, targets_test = targets[0], targets[1]

    c = features_train.shape[0] // optimization_specification['batch_size']
    keys = tree_map(lambda x: x * c, np.arange(50, 1000, 50))
    values = [0.5, ] * len(keys)
    dict_lr = dict(zip(keys, values))

    sc = optax.piecewise_constant_schedule(optimization_specification['learning_rate'], dict_lr)
    optimizer = optax.adamw(sc, weight_decay=1e-2)
    opt_state = optimizer.init(eqx.filter(model, eqx.is_array))

    make_step = lambda x, y: optimization_specification['make_step'](x, y, optimizer)

    for it in tqdm(range(optimization_specification['N_epochs'])):
        key = random.PRNGKey(it)
        loss, model, opt_state = train_on_epoch(key, optimization_specification['batch_size'], model, features_train, coordinates, targets_train, opt_state, make_step)
        history_train.append(jnp.mean(loss))

        test_loss = test_on_epoch(key, optimization_specification['batch_size'], model, features_test, coordinates, targets_test)
        history_test.append(test_loss)
    return model, history_train, history_test

def get_datasets(direction_to_save, type_of_pde, grid):
    train_dataset, test_dataset = load_ConDiff(save_dir=direction_to_save, 
                                               pde=type_of_pde, 
                                               grid=grid)
    x = jnp.linspace(0, 1, grid)
    y = jnp.linspace(0, 1, grid)
    xx, yy = jnp.meshgrid(x, y)
    coordinates = jnp.stack([xx, yy], 0)
    
    features_train = tree_map(lambda x: x.reshape(-1, grid, grid), train_dataset[0])
    targets_train = tree_map(lambda x: x.reshape(-1, grid, grid), train_dataset[1])
    
    features_test = tree_map(lambda x: x.reshape(-1, grid, grid), test_dataset[0])
    targets_test = tree_map(lambda x: x.reshape(-1, grid, grid), test_dataset[1])
    
    return [jnp.expand_dims(features_train, 1), jnp.expand_dims(targets_train, 1)], \
           [jnp.expand_dims(features_test, 1), jnp.expand_dims(targets_test, 1)], coordinates

In [9]:
def get_results(key, direction_to_save, type_of_pde, grid):
    keys = random.split(key, 3)
    
    train_dataset, test_dataset, coordinates = get_datasets(direction_to_save, type_of_pde, grid)
    features_train, targets_train = train_dataset[0], train_dataset[1]
    features_test, targets_test = test_dataset[0], test_dataset[1]

    model_data, optimization_specification = get_FFNO(keys[0], coordinates, features_train, grid)
    
    features = [features_train, features_test]
    targets = [targets_train, targets_test]
    
    model, history_train, history_test = train_model(model_data, features, coordinates, targets, 
                                                 optimization_specification=optimization_specification)
    
    N_samples_train = features_train.shape[0]
    list_of_indices_train = jnp.linspace(0, N_samples_train-1, N_samples_train, dtype=jnp.int64)
    n_batches_train = N_samples_train // optimization_specification['batch_size']
    n_train = random.choice(keys[1], list_of_indices_train, shape = (n_batches_train, optimization_specification['batch_size']))
    
    N_samples_test = features_test.shape[0]
    list_of_indices_test = jnp.linspace(0, N_samples_test-1, N_samples_test, dtype=jnp.int64)
    n_batches_test = N_samples_test // optimization_specification['batch_size']
    n_test = random.choice(keys[2], list_of_indices_test, shape = (n_batches_test, optimization_specification['batch_size']))
    
    loss_train_error = lambda a, x: (None, compute_relative_error(model, features_train, targets_train, coordinates, x))
    _, train_loss_error = scan(loss_train_error, None, n_train)
    train_error_mean = jnp.mean(train_loss_error.reshape(-1,))
    
    loss_test_error = lambda a, x: (None, compute_relative_error(model, features_test, targets_test, coordinates, x))
    _, test_loss_error = scan(loss_test_error, None, n_test)
    test_error_mean = jnp.mean(test_loss_error.reshape(-1,))

    train_error_std = jnp.sqrt(jnp.var(test_loss_error.reshape(-1,)))
    test_error_std = jnp.sqrt(jnp.var(test_loss_error.reshape(-1,)))

    model_size = sum(tree_map(jnp.size, tree_flatten(model)[0], is_leaf=eqx.is_array))
    
    data = {
        'history_train': history_train,
        'history_test': history_test,
        'train_error_mean': train_error_mean,
        'test_error_mean': test_error_mean,
        'train_error_std': train_error_std,
        'test_error_std': test_error_std,
        'model_size': model_size
    }
    return data, model, model_data, features_test, targets_test, coordinates

In [ ]:
key = random.PRNGKey(22)

grid = 64
type_of_pde = 'poisson'
direction_to_save = 

data, model, model_data, features_test, targets_test, coordinates = get_results(key, direction_to_save, type_of_pde, grid)

2024-06-05 17:50:58.978304: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 398/400 [14:53<00:04,  2.22s/it]

In [ ]:
def plot_results(model, model_data, data, features_test, targets_test, coordinates):
    prediction = vmap(lambda x, coords: model(x, coords), in_axes=(0, None))(features_test, coordinates)[:, 0]

    display.clear_output(wait=True)

    plt.rcParams['font.family'] = 'serif'
    fig, ax = plt.subplots(1, len(prediction[0].shape)+1, figsize=((len(prediction[0].shape)+1)*5, 4))

    ax[0].set_title(r'Loss', fontsize="15")
    ax[0].set_yscale('log')
    ax[0].set_xlabel(r'# its.')
    ax[0].plot(data['history_train'], linestyle='-', color='red', label='train')
    ax[0].plot(data['history_test'], linestyle='-.', color='green', label='test')
    ax[0].legend(loc='best', fontsize="12")
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['left'].set_visible(True)
    ax[0].spines['right'].set_visible(False)
    ax[0].grid(linestyle='-.')

    ax[1].contourf(prediction[0])
    ax[1].set_title(r'Prediction')

    ax[2].contourf(targets_test[0, 0])
    ax[2].set_title(r'Target')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_results(model, model_data, data, features_test, targets_test, coordinates)

In [ ]:
model_size = data['model_size']
train_error_mean = jnp.round(data['train_error_mean'], 3)
test_error_mean = jnp.round(data['test_error_mean'], 3)
train_error_std = jnp.round(data['train_error_std'], 3)
test_error_std = jnp.round(data['test_error_std'], 3)

print(f'Model size: {model_size}')
print(f"Train relative error: {train_error_mean} ± {train_error_std}, \nTest relative error: {test_error_mean} ± {test_error_std}")